In [1]:
from collections import defaultdict
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
from scipy import stats
import statsmodels.api as sm
from tqdm import tqdm

from plotting import *

In [2]:
results_dir = 'results/org_analysis/Sep18/'
os.makedirs(results_dir, exist_ok=True)

filepath = "data/ML Systems-Org view.csv"

cutoff_date = '2018-01-01'

In [3]:
# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Functions

In [4]:
def is_open(accessibility):
    return accessibility.startswith('Open') if isinstance(accessibility, str) else False


def compare_org_openness_with_compute(df, grouping_column):
    df = df[pd.to_datetime(df['Publication date']) >= pd.to_datetime(cutoff_date)].copy()
    df.loc[:, 'is_open_model'] = df['Model accessibility'].apply(is_open)
    df.loc[:, 'is_open_code'] = df['Code accessibility'].apply(is_open)
    df.loc[:, 'log_compute'] = np.log10(df['Training compute (FLOP)'])

    # Group by organization and get the stats
    org_stats = df.groupby(grouping_column).agg({
        'log_compute': ['max', 'mean'],
        'is_open_model': ['mean'],
        'is_open_code': ['mean', 'count'],
    }).reset_index()

    # Flatten the multi-level column index and rename columns
    org_stats.columns = [grouping_column, 'Max Training Compute (FLOP)', 'Average Training Compute (FLOP)', 'open_model_percentage', 'open_code_percentage', 'model_count']
    org_stats['open_model_percentage'] = org_stats['open_model_percentage'] * 100
    org_stats['open_code_percentage'] = org_stats['open_code_percentage'] * 100

    # Sort by the most compute-intensive model (descending) and reset index
    result = org_stats.sort_values('Max Training Compute (FLOP)', ascending=False).reset_index(drop=True)
    return result

In [5]:
def bootstrap_openness(df, grouping_column, n_iterations=10000):
    results = defaultdict(list)
    
    for _ in tqdm(range(n_iterations)):
        # Resample with replacement
        bootstrap_sample = df.sample(n=len(df), replace=True)
        
        # Calculate openness for this sample
        sample_results = compare_org_openness_with_compute(bootstrap_sample, grouping_column)
        
        for _, row in sample_results.iterrows():
            results[row[grouping_column]].append({
                'open_model_percentage': row['open_model_percentage'],
                'open_code_percentage': row['open_code_percentage']
            })
    
    return results


def compare_entities_on_openness(results, entity1, entity2, metric='open_model'):
    values1 = [v[f'{metric}_percentage'] for v in results[entity1]]
    values2 = [v[f'{metric}_percentage'] for v in results[entity2]]
    values1_ci = np.percentile(values1, [5, 95])
    values2_ci = np.percentile(values2, [5, 95])
    _, p_value = stats.ttest_ind(values1, values2)

    print(f"{metric} percentage")
    print(f"{entity1}: {np.mean(values1):.2f}% (90% CI: {values1_ci[0]:.2f} to {values1_ci[1]:.2f})")
    print(f"{entity2}: {np.mean(values2):.2f}% (90% CI: {values2_ci[0]:.2f} to {values2_ci[1]:.2f})")
    print(f"p-value for difference between {entity1} and {entity2}: {p_value}")

# Openness by affiliation

In [6]:
df = pd.read_csv(filepath)

In [7]:
df.head()

,System,Publication date,Notability criteria,Link,Model accessibility,Code accessibility,Training dataset,Dataset accessibility,Accessibility notes,Organization,Training compute (FLOP),Country (from Organization),Organization categorization
0,Grok-2,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
1,Grok-2 mini,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
2,Cosine Genie,2024-08-12,NaN,https://cosine.sh/blog/genie-technical-report,Unreleased,NaN,Unspecified unreleased,NaN,https://github.com/CosineAI/experiments/tree/c...,Cosine,NaN,United Kingdom of Great Britain and Northern I...,Industry
3,EXAONE 3.0,2024-08-07,NaN,https://arxiv.org/abs/2408.03541,Open access (non-commercial),NaN,Unspecified unreleased,NaN,NaN,LG,4.000000e+23,Korea (Republic of),Industry
4,Flux.1 [pro],2024-08-01,NaN,https://blackforestlabs.ai/announcing-black-fo...,API access,Unreleased,NaN,Unreleased,NaN,Black Forest Labs,NaN,NaN,NaN


In [8]:
df['Organization'].unique()

array(['xAI', 'Cosine', 'LG', 'Black Forest Labs', 'Apple',
       'Google DeepMind', 'Mistral AI', 'Meta AI', 'OpenAI', 'Alibaba',
       'EvolutionaryScale,UC Berkeley', 'Anthropic', 'Zhipu AI',
       'Zhipu AI,Tsinghua University', 'DeepSeek', 'Runway', 'NVIDIA',
       'Stanford University,UC Berkeley,Toyota Research Institute,Google DeepMind,Massachusetts Institute of Technology (MIT),Physical Intelligence',
       'Microsoft,University of Illinois Urbana-Champaign (UIUC)',
       'Facebook AI Research', 'Columbia University,Rutgers University',
       '01.AI',
       'Tohoku University,CyberAgent,Tokyo Institute of Technology,Fujitsu,RIKEN,Nagoya University,Kotoba Technologies',
       'Technology Innovation Institute', 'Tokyo Institute of Technology',
       'Microsoft', 'SenseTime', 'Sber', 'Reka AI',
       'NTT Communication Science Laboratories', 'Stability AI', 'Yandex',
       'Cohere,Cohere for AI', 'Silo AI,University of Turku',
       'Google DeepMind,McGill University

In [9]:
{**df.iloc[1]}

{'System': 'Grok-2 mini',
 'Publication date': '2024-08-13',
 'Notability criteria': nan,
 'Link': 'https://x.ai/blog/grok-2',
 'Model accessibility': 'Hosted access (no API)',
 'Code accessibility': 'Unreleased',
 'Training dataset': nan,
 'Dataset accessibility': nan,
 'Accessibility notes': nan,
 'Organization': 'xAI',
 'Training compute (FLOP)': nan,
 'Country (from Organization)': 'United States of America',
 'Organization categorization': 'Industry'}

In [10]:
aliases = {
  'Beijing Academy of Artificial Intelligence / BAAI': 'BAAI',
  'Google': 'Google/DeepMind',
  'Google Research': 'Google/DeepMind',
  'DeepMind': 'Google/DeepMind',
  'Google Brain': 'Google/DeepMind',
  'Google DeepMind': 'Google/DeepMind',
  'Facebook AI Research': 'Meta/Facebook',
  'Facebook AI Research': 'Meta/Facebook',
  'Facebook': 'Meta/Facebook',
  'Facebook AI': 'Meta/Facebook',
  'Meta AI': 'Meta/Facebook',
  'Meta AI Research': 'Meta/Facebook',
  'Massachusetts Institute of Technology (MIT)': 'MIT',
  'Stanford University': 'Stanford',
  'New York University (NYU)': 'NYU',
  'Carnegie Mellon University (CMU)': 'CMU',
  'University of Washington': 'UoW',
  'UC Berkeley': 'UCB',
  'Tsinghua University': 'Tsinghua',
}

new_rows = []
for i, row in df.dropna(subset=['Organization']).iterrows():
  orgs = row['Organization']
  orgs = orgs.split(',')
  for org in orgs:
    if org in aliases:
      org = aliases[org]
    new_row = {**row}
    new_row['Organization'] = org
    new_rows.append(new_row)

df = pd.DataFrame(new_rows)

In [11]:
df.head()

,System,Publication date,Notability criteria,Link,Model accessibility,Code accessibility,Training dataset,Dataset accessibility,Accessibility notes,Organization,Training compute (FLOP),Country (from Organization),Organization categorization
0,Grok-2,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
1,Grok-2 mini,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
2,Cosine Genie,2024-08-12,NaN,https://cosine.sh/blog/genie-technical-report,Unreleased,NaN,Unspecified unreleased,NaN,https://github.com/CosineAI/experiments/tree/c...,Cosine,NaN,United Kingdom of Great Britain and Northern I...,Industry
3,EXAONE 3.0,2024-08-07,NaN,https://arxiv.org/abs/2408.03541,Open access (non-commercial),NaN,Unspecified unreleased,NaN,NaN,LG,4.000000e+23,Korea (Republic of),Industry
4,Flux.1 [pro],2024-08-01,NaN,https://blackforestlabs.ai/announcing-black-fo...,API access,Unreleased,NaN,Unreleased,NaN,Black Forest Labs,NaN,NaN,NaN


In [12]:
len(df)

2601

In [13]:
df.dropna(subset=['Notability criteria'], inplace=True)
len(df)

1407

In [14]:
all_models = compare_org_openness_with_compute(df, 'Organization')

In [15]:
len(all_models)

225

In [16]:
all_models.head(40)

,Organization,Max Training Compute (FLOP),Average Training Compute (FLOP),open_model_percentage,open_code_percentage,model_count
0,Google/DeepMind,25.698970,21.747013,30.405405,35.135135,148
1,Meta/Facebook,25.579784,21.322178,78.823529,63.529412,85
2,OpenAI,25.322219,22.424921,34.482759,20.689655,29
3,NVIDIA,25.255273,22.366720,46.153846,23.076923,13
4,Peking University,25.079181,22.523616,0.000000,66.666667,6
5,ByteDance,25.079181,24.147339,0.000000,66.666667,3
6,Mistral AI,25.049218,25.049218,50.000000,0.000000,2
7,Inflection AI,25.000434,25.000239,0.000000,0.000000,2
8,Anthropic,24.587262,24.587262,0.000000,0.000000,5
9,Technology Innovation Institute,24.575188,23.977700,100.000000,0.000000,2


In [17]:
len(all_models[all_models['open_model_percentage'] > 99]['Organization'])

82

In [18]:
len(all_models[all_models['open_model_percentage'] < 1]['Organization'])

83

In [19]:
all_models.dropna(subset=['Max Training Compute (FLOP)'], inplace=True)
len(all_models)

179

In [20]:
# Plot distributions
fig = px.histogram(all_models, x='Max Training Compute (FLOP)', nbins=50, title='Distribution of log-FLOP')
fig.show()
fig = px.histogram(all_models, x='open_model_percentage', nbins=50, title='Distribution of open model percentage')
fig.show()
fig = px.histogram(all_models, x='open_code_percentage', nbins=50, title='Distribution of open code percentage')
fig.show()
fig = px.histogram(all_models, x='model_count', nbins=50, title='Distribution of model count')
fig.show()

In [21]:
# for accessibility_type in ['model', 'code']:
#     for exclude_meta_facebook in [False, True]:
#         print(f"\n# {accessibility_type} accessibility {'without Meta/Facebook' if exclude_meta_facebook else ''}\n")
#         for model_count_threshold in [1, 3, *range(5, 55, 5)]:
#             big_org_models = all_models[all_models['model_count'] >= model_count_threshold]

#             if exclude_meta_facebook:
#                 big_org_models = big_org_models[big_org_models['Organization'] != 'Meta/Facebook']

#             if len(big_org_models) < 3:
#                 continue

#             # Perform a permutation test to see if there is significant correlation
#             x = big_org_models['Max Training Compute (FLOP)']
#             y = big_org_models[f'open_{accessibility_type}_percentage']
#             corr = stats.spearmanr(x, y).statistic
#             dof = len(y) - 2
#             def statistic(x):  # explore all possible pairings by permuting `x`
#                 rs = stats.spearmanr(x, y).statistic  # ignore pvalue
#                 transformed = rs * np.sqrt(dof / ((rs+1.0)*(1.0-rs)))
#                 return transformed
#             ref = stats.permutation_test((x,), statistic, alternative='greater', permutation_type='pairings')

#             print(f"threshold={model_count_threshold} n={len(big_org_models)} corr={corr:.3f} statistic={ref.statistic:.3f} p={ref.pvalue:.3f}")

In [28]:
accessibility_type = 'model'
model_count_threshold = 15

fig = make_subplots(rows=1, cols=2)
big_org_models = all_models[all_models['model_count'] >= model_count_threshold]
other_org_models = all_models[all_models['model_count'] < model_count_threshold]

fig.add_trace(
    go.Scatter(
        x=other_org_models['Max Training Compute (FLOP)'],
        y=other_org_models[f'open_{accessibility_type}_percentage'],
        mode='markers',
        text=other_org_models['Organization'],
        marker=dict(size=10, color='#daf0f0'),
        showlegend=False,
    ),
    row=1,
    col=1,
)

# Add trendline to first subplot using statsmodels
# if len(all_models) >= 2:
#     X1 = sm.add_constant(all_models['Max Training Compute (FLOP)'])
#     model1 = sm.OLS(all_models[f'open_{accessibility_type}_percentage'], X1).fit()
#     x_range1 = np.linspace(all_models['Max Training Compute (FLOP)'].min(), all_models['Max Training Compute (FLOP)'].max(), 100)
#     X_pred1 = sm.add_constant(x_range1)
#     y_fit1 = model1.predict(X_pred1)
#     fig.add_trace(
#         go.Scatter(
#             x=x_range1,
#             y=y_fit1,
#             mode='lines',
#             name='Trendline',
#             line=dict(color='rgb(150, 150, 150)'),
#             showlegend=False,
#         ),
#         row=1,
#         col=1,
#     )

fig.add_trace(
    go.Scatter(
        x=big_org_models['Max Training Compute (FLOP)'],
        y=big_org_models[f'open_{accessibility_type}_percentage'],
        mode='markers+text',
        marker=dict(
            size=10, #2*big_org_models['model_count']**0.5,
            color='#00a5a6',
            opacity=1,
        ),
        text=big_org_models['Organization'],
        textposition='top center',
        showlegend=False,
    ),
    row=1,
    col=1,
)

# x limits
fig.update_xaxes({'range': [14, 28]}, row=1, col=1)
# xaxis title
accessibility_obj = 'models' if accessibility_type == 'model' else 'code'
fig.update_xaxes(title='Largest training run (FLOP, log-scale)', row=1, col=1)
fig.update_yaxes(title=f'Percentage of open {accessibility_obj}', row=1, col=1)
fig.update_yaxes(
    range=[-10, 110],
    tickmode='array',
    tickvals=[-10, 0, 25, 50, 75, 100, 110],
    ticktext=['', '0', '25', '50', '75', '100', ''],
    row=1,
    col=1,
)
# fig.update_yaxes(range=[0, 100], row=1, col=1)

fig.add_trace(
    go.Scatter(
        x=np.log10(other_org_models['model_count']),
        y=other_org_models[f'open_{accessibility_type}_percentage'],
        mode='markers',
        text=other_org_models['Organization'],
        marker=dict(size=10, color='#daf0f0'),
        showlegend=False,
    ),
    row=1,
    col=2,
)

# Add trendline to second subplot using statsmodels
# if len(all_models) >= 2:
#     X2 = sm.add_constant(np.log10(all_models['model_count']))
#     model2 = sm.OLS(all_models[f'open_{accessibility_type}_percentage'], X2).fit()
#     x_range2 = np.linspace(np.log10(all_models['model_count']).min(), np.log10(all_models['model_count']).max(), 100)
#     X_pred2 = sm.add_constant(x_range2)
#     y_fit2 = model2.predict(X_pred2)
#     fig.add_trace(
#         go.Scatter(
#             x=x_range2,
#             y=y_fit2,
#             mode='lines',
#             name='Trendline',
#             line=dict(color='rgb(150, 150, 150)'),
#             showlegend=False,
#         ),
#         row=1,
#         col=2,
#     )

fig.add_trace(
    go.Scatter(
        x=np.log10(big_org_models['model_count']),
        y=big_org_models[f'open_{accessibility_type}_percentage'],
        mode='markers+text',
        marker=dict(size=10, color='#00a5a6'),
        text=big_org_models['Organization'],
        textposition='top center',
        showlegend=False,
    ),
    row=1,
    col=2,
)

fig.update_xaxes(title='Number of notable models (log-scale)', range=[-0.5, 3], row=1, col=2)
# Edit tick labels
fig.update_xaxes(
    tickmode='array',
    tickvals=np.log10([1, 3, 10, 30, 100, 300]),
    ticktext=['1', '3', '10', '30', '100', '300'],
    row=1,
    col=2,
)
fig.update_yaxes(
    range=[-10, 110],
    tickmode='array',
    tickvals=[-10, 0, 25, 50, 75, 100, 110],
    ticktext=['', '0', '25', '50', '75', '100', ''],
    row=1,
    col=2,
)
# fig.update_yaxes(range=[0, 100], row=1, col=2)

# Update layout for better readability
fig.update_layout(
  width=1000,
  height=500,
  template='plotly_white',
  coloraxis_showscale=False,
  title=f"Percentage of notable models that are open, by organization",
)

# Save plot
save_plot(fig, results_dir, f'percent_open_{accessibility_type}_by_org')

# Show the plot
fig.show()

In [ ]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     open_model_percentage   R-squared:                       0.060
Model:                               OLS   Adj. R-squared:                  0.055
Method:                    Least Squares   F-statistic:                     11.26
Date:                   Wed, 18 Sep 2024   Prob (F-statistic):           0.000969
Time:                           16:58:25   Log-Likelihood:                -918.07
No. Observations:                    179   AIC:                             1840.
Df Residuals:                        177   BIC:                             1847.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                         -45.0012     30.119     -1.494      0.137    -104.439      14.437
Max Training Compute (FLOP)     4.6587      1.388      3.356      0.001       1.919       7.399
==============================================================================
Omnibus:                      432.697   Durbin-Watson:                   1.312
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               15.640
Skew:                          -0.144   Prob(JB):                     0.000402
Kurtosis:                       1.581   Cond. No.                         213.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     open_model_percentage   R-squared:                       0.004
Model:                               OLS   Adj. R-squared:                 -0.001
Method:                    Least Squares   F-statistic:                    0.7543
Date:                   Wed, 18 Sep 2024   Prob (F-statistic):              0.386
Time:                           16:58:27   Log-Likelihood:                -923.21
No. Observations:                    179   AIC:                             1850.
Df Residuals:                        177   BIC:                             1857.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          57.6911      4.015     14.368      0.000      49.767      65.615
model_count    -6.7237      7.742     -0.869      0.386     -22.001       8.554
==============================================================================
Omnibus:                     2354.789   Durbin-Watson:                   1.235
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.039
Skew:                          -0.291   Prob(JB):                     2.70e-05
Kurtosis:                       1.425   Cond. No.                         2.74
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Openness by Organization categorization

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
df.head()

,System,Publication date,Notability criteria,Link,Model accessibility,Code accessibility,Training dataset,Dataset accessibility,Accessibility notes,Organization,Training compute (FLOP),Country (from Organization),Organization categorization
0,Grok-2,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
1,Grok-2 mini,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
2,Cosine Genie,2024-08-12,NaN,https://cosine.sh/blog/genie-technical-report,Unreleased,NaN,Unspecified unreleased,NaN,https://github.com/CosineAI/experiments/tree/c...,Cosine,NaN,United Kingdom of Great Britain and Northern I...,Industry
3,EXAONE 3.0,2024-08-07,NaN,https://arxiv.org/abs/2408.03541,Open access (non-commercial),NaN,Unspecified unreleased,NaN,NaN,LG,4.000000e+23,Korea (Republic of),Industry
4,Flux.1 [pro],2024-08-01,NaN,https://blackforestlabs.ai/announcing-black-fo...,API access,Unreleased,NaN,Unreleased,NaN,Black Forest Labs,NaN,NaN,NaN


In [ ]:
df['Organization categorization'].unique()

array(['Industry', nan, 'Industry,Academia',
       'Academia,Academia,Industry,Industry,Academia,Industry',
       'Academia,Academia',
       'Academia,Industry,Academia,Industry,Academia,Industry',
       'Government', 'Academia', 'Industry,Industry',
       'Industry,Academia,Academia', 'Academia,Industry',
       'Industry,Industry,Industry', 'Academia,Academia,Industry',
       'Industry,Academia,Industry,Academia,Academia',
       'Research collective,Academia',
       'Academia,Industry,Academia,Academia',
       'Industry,Academia,Academia,Academia',
       'Industry,Academia,Industry', 'Government,Industry',
       'Academia,Academia,Academia', 'Industry,Government',
       'Academia,Industry,Academia',
       'Academia,Academia,Academia,Academia',
       'Academia,Academia,Research collective,Academia,Academia,Industry,Academia',
       'Research collective',
       'Industry,Industry,Academia,Academia,Academia,Academia,Academia,Academia,Industry,Academia,Academia,Government

In [ ]:
new_rows = []
for i, row in df.dropna(subset=['Organization categorization']).iterrows():
  org_types = row['Organization categorization']
  org_types = org_types.split(',')
  for org_type in org_types:
    print(org_type)
    new_row = {**row}
    new_row['Organization categorization'] = org_type
    new_rows.append(new_row)

df = pd.DataFrame(new_rows)

Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Academia
Industry
Academia
Industry
Industry
Industry
Industry
Industry
Industry
Academia
Industry
Industry
Industry
Academia
Academia
Industry
Industry
Academia
Industry
Industry
Industry
Academia
Industry
Industry
Industry
Academia
Academia
Industry
Industry
Industry
Industry
Academia
Industry
Academia
Industry
Academia
Industry
Industry
Government
Industry
Industry
Industry
Academia
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Academia
Industry
Academia
Academia
Academia
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Academia
Industry
Industry
Industry
Industry
Industry
Industry
Academia
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry
Industry

In [ ]:
df['Organization categorization'].unique()

array(['Industry', 'Academia', 'Government', 'Research collective'],
      dtype=object)

In [ ]:
len(df)

2541

In [ ]:
df.dropna(subset=['Notability criteria'], inplace=True)
len(df)

1384

In [ ]:
all_models = compare_org_openness_with_compute(df, 'Organization categorization')

In [ ]:
all_models.head(40)

,Organization categorization,Max Training Compute (FLOP),Average Training Compute (FLOP),open_model_percentage,open_code_percentage,model_count
0,Industry,25.698970,21.615142,45.945946,38.063063,444
1,Academia,25.079181,20.686575,52.528090,49.157303,356
2,Government,24.575188,22.256482,91.666667,8.333333,12
3,Research collective,23.761176,20.454918,50.000000,50.000000,14


In [ ]:
categorization_bootstrap = bootstrap_openness(df, 'Organization categorization')

100%|██████████| 10000/10000 [00:24<00:00, 405.45it/s]


In [ ]:
category1 = 'Industry'
category2 = 'Academia'
compare_entities_on_openness(categorization_bootstrap, category1, category2, 'open_model')
compare_entities_on_openness(categorization_bootstrap, category1, category2, 'open_code')

open_model percentage
Industry: 45.92% (90% CI: 42.02 to 49.78)
Academia: 52.50% (90% CI: 48.19 to 56.81)
p-value for difference between Industry and Academia: 0.0
open_code percentage
Industry: 38.04% (90% CI: 34.27 to 41.91)
Academia: 49.16% (90% CI: 44.80 to 53.46)
p-value for difference between Industry and Academia: 0.0


# Openness by Country

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
df.head()

,System,Publication date,Notability criteria,Link,Model accessibility,Code accessibility,Training dataset,Dataset accessibility,Accessibility notes,Organization,Training compute (FLOP),Country (from Organization),Organization categorization
0,Grok-2,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
1,Grok-2 mini,2024-08-13,NaN,https://x.ai/blog/grok-2,Hosted access (no API),Unreleased,NaN,NaN,NaN,xAI,NaN,United States of America,Industry
2,Cosine Genie,2024-08-12,NaN,https://cosine.sh/blog/genie-technical-report,Unreleased,NaN,Unspecified unreleased,NaN,https://github.com/CosineAI/experiments/tree/c...,Cosine,NaN,United Kingdom of Great Britain and Northern I...,Industry
3,EXAONE 3.0,2024-08-07,NaN,https://arxiv.org/abs/2408.03541,Open access (non-commercial),NaN,Unspecified unreleased,NaN,NaN,LG,4.000000e+23,Korea (Republic of),Industry
4,Flux.1 [pro],2024-08-01,NaN,https://blackforestlabs.ai/announcing-black-fo...,API access,Unreleased,NaN,Unreleased,NaN,Black Forest Labs,NaN,NaN,NaN


In [ ]:
df['Country (from Organization)'].unique()

array(['United States of America',
       'United Kingdom of Great Britain and Northern Ireland',
       'Korea (Republic of)', nan, 'Multinational', 'France', 'China',
       'United States of America,United States of America', 'China,China',
       'United States of America,United States of America,United States of America,Multinational,United States of America,United States of America',
       'Japan,Japan,Japan,Japan,Japan,Japan', 'United Arab Emirates',
       'Japan', 'Hong Kong', 'Russia', 'Canada,Multinational',
       'Finland,Finland', 'Multinational,Canada,Canada', 'Israel',
       'Multinational,United States of America,United States of America',
       'United States of America,United States of America,United States of America',
       'Multinational,United States of America,Canada,United States of America,United States of America',
       'United States of America,China,Singapore,United States of America',
       'Multinational,Israel,Israel,Israel',
       'Multinational

In [ ]:
new_rows = []
for i, row in df.dropna(subset=['Country (from Organization)']).iterrows():
  countries = row['Country (from Organization)']
  countries = countries.split(',')
  for country in countries:
    print(country)
    new_row = {**row}
    new_row['Country (from Organization)'] = country
    new_rows.append(new_row)

df = pd.DataFrame(new_rows)

United States of America
United States of America
United Kingdom of Great Britain and Northern Ireland
Korea (Republic of)
United States of America
United States of America
Multinational
Multinational
France
United States of America
United States of America
United States of America
France
United States of America
China
United States of America
United States of America
United States of America
United States of America
Multinational
Multinational
Multinational
United States of America
China
China
China
China
United States of America
United States of America
United States of America
United States of America
United States of America
Multinational
United States of America
United States of America
United States of America
United States of America
United States of America
China
United States of America
France
United States of America
United States of America
United States of America
United States of America
China
United States of America
Japan
Japan
Japan
Japan
Japan
Japan
Multinational
Unite

In [ ]:
df['Country (from Organization)'].unique()

array(['United States of America',
       'United Kingdom of Great Britain and Northern Ireland',
       'Korea (Republic of)', 'Multinational', 'France', 'China', 'Japan',
       'United Arab Emirates', 'Hong Kong', 'Russia', 'Canada', 'Finland',
       'Israel', 'Singapore', 'Switzerland', 'Austria', 'Saudi Arabia',
       'Hungary', 'Denmark', 'Germany', 'Australia', 'Sweden', 'India',
       'Spain', 'Netherlands', 'Greece', 'Viet Nam', 'Portugal', 'Taiwan',
       'Lithuania', 'Lebanon', 'Norway', 'Belgium', 'Ireland', 'Poland',
       'Czechia', 'Iran (Islamic Republic of)', 'Argentina', 'Italy',
       'Mexico', 'New Zealand'], dtype=object)

In [ ]:
len(df['Country (from Organization)'].unique())

41

In [ ]:
len(df)

2576

In [ ]:
df.dropna(subset=['Notability criteria'], inplace=True)
len(df)

1406

In [ ]:
all_models = compare_org_openness_with_compute(df, 'Country (from Organization)')

In [ ]:
all_models.head(40)

,Country (from Organization),Max Training Compute (FLOP),Average Training Compute (FLOP),open_model_percentage,open_code_percentage,model_count
0,Multinational,25.698970,22.119227,53.448276,37.931034,58
1,United States of America,25.579784,21.203979,50.947368,44.842105,475
2,China,25.079181,21.526964,45.535714,47.321429,112
3,France,25.049218,20.754704,70.588235,58.823529,17
4,United Arab Emirates,24.575188,23.481317,100.000000,0.000000,3
5,United Kingdom of Great Britain and Northern I...,23.800029,20.691535,21.568627,25.490196,51
6,Canada,23.633468,21.039008,40.909091,50.000000,22
7,Korea (Republic of),23.169086,21.787130,47.368421,15.789474,19
8,Israel,22.927370,21.000010,75.000000,50.000000,8
9,India,22.927370,20.385979,66.666667,0.000000,3


In [ ]:
country_bootstrap = bootstrap_openness(df, 'Country (from Organization)')

100%|██████████| 10000/10000 [00:27<00:00, 364.84it/s]


In [ ]:
country1 = 'United States of America'
country2 = 'China'
compare_entities_on_openness(country_bootstrap, country1, country2, 'open_model')
compare_entities_on_openness(country_bootstrap, country1, country2, 'open_code')

open_model percentage
United States of America: 50.99% (90% CI: 47.13 to 54.85)
China: 45.43% (90% CI: 37.61 to 53.27)
p-value for difference between United States of America and China: 0.0
open_code percentage
United States of America: 44.90% (90% CI: 41.12 to 48.66)
China: 47.26% (90% CI: 39.37 to 54.95)
p-value for difference between United States of America and China: 0.0


In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

# Create a 1x2 subplot for the '% open weights' and '% open training code'
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('% Open Weights by Model Affiliation', '% Open Training Code by Model Affiliation'))

# '% open weights' bar chart
fig.add_trace(go.Bar(x=df["Model affiliation"], y=df["% open weights"], name='% Open Weights'), row=1, col=1)

# '% open training code' bar chart
fig.add_trace(go.Bar(x=df["Model affiliation"], y=df["% open training code"], name='% Open Training Code'), row=1, col=2)

# Update layout for better visibility
fig.update_layout(height=500, width=1000, showlegend=False, template='plotly_white')
fig.update_xaxes(tickangle=45)

save_plot(fig, results_dir, 'percent_open_weights_and_code_by_affiliation')

# Display the plot
fig.show()